In [5]:
from torch_geometric.loader import DataLoader
import torch
from models.special_supervised import VGAEGCN
from utils.dataset import split_dataset

data_train, data_test, data_val, = split_dataset(True)
model = VGAEGCN(1, 1, 32, molecular=True, trans=True)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

train_loader = DataLoader(data_train, batch_size=10, shuffle=True)
test_loader = DataLoader(data_test, batch_size=10)



In [6]:
import torch.nn.functional as F


def train(model, train_loader, optimizer):
    model.train()
    for data in train_loader:
        optimizer.zero_grad()
        out, _ = model(data)
        loss = F.l1_loss(out, data.y)
        loss.backward()
        optimizer.step()

# Define the test loop
def v_test(model, loader):
    model.eval()
    l1_loss = 0.0
    mse_loss = 0.0
    for data in loader:
        out, _ = model(data)
        mse_loss += F.mse_loss(out, data.y)
        l1_loss += F.l1_loss(out, data.y)
    return (mse_loss / len(loader), l1_loss / len(loader))


In [8]:
for epoch in range(1, 5):
    train(model, train_loader, optimizer)
    acc = v_test(model, test_loader)
    print(f'Epoch: {epoch:03d}, MSE: {acc[0]:.4f}, L1: {acc[1]:.4f}')


Epoch: 001, MSE: 39.0208, L1: 14.9696

Epoch: 002, MSE: 40.8902, L1: 14.7864

Epoch: 003, MSE: 40.7075, L1: 14.7534



KeyboardInterrupt: 